## Problem statement
- A large amount of movie rating data is given to build an RBM. The task is to predict if a user likes a movie as 1 or dislike as 0. Recomment Movies that user may like.

### Import required libraries

In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Load, Movie, Ratings and Users Dataset

In [2]:
movies = pd.read_csv('https://raw.githubusercontent.com/luke4u/Movie-Recommender/master/ml-1m/movies.dat',
                     sep = '::',header = None, engine = 'python',
                     encoding = 'latin-1')
users =pd.read_csv('https://raw.githubusercontent.com/luke4u/Movie-Recommender/master/ml-1m/users.dat',
                   sep = '::', header = None, engine = 'python',
                   encoding = 'latin-1')
ratings =pd.read_csv('https://raw.githubusercontent.com/luke4u/Movie-Recommender/master/ml-1m/ratings.dat',
                     sep = '::', header = None, engine = 'python',
                     encoding = 'latin-1')

In [3]:
movies[:5]

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users[:5]

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings[:5]

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [53]:
movies.columns=["movie_id", "movie_name", "genre"]
movies.head()

,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.columns=["user_id", "movie_id", "rating", "time_stamp"]
ratings.head()

,user_id,movie_id,rating,time_stamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
users.columns=['user_id', 'gender', 'age', 'job_code','zip_code']
users.head()

,user_id,gender,age,job_code,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


### Load training and testing Data

In [9]:
train = pd.read_csv('https://raw.githubusercontent.com/luke4u/Movie-Recommender/master/ml-1m/training_set.csv', delimiter = ',')
train = np.array(train, dtype = 'int')
test = pd.read_csv('https://raw.githubusercontent.com/luke4u/Movie-Recommender/master/ml-1m/test_set.csv', delimiter = ',')
test = np.array(test, dtype = 'int')

In [10]:
train_df=train.copy()
train_df=pd.DataFrame(train_df, columns=['user_id','movie_id','rating','time_stamp'])
train_df.head()

,user_id,movie_id,rating,time_stamp
0,1,661,3,978302109
1,1,914,3,978301968
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1287,5,978302039


In [11]:
test_df=test.copy()
test_df=pd.DataFrame(test_df, columns=['user_id','movie_id','rating','time_stamp'])
test_df.head()

,user_id,movie_id,rating,time_stamp
0,1,1193,5,978300760
1,1,1197,3,978302268
2,1,2804,5,978300719
3,1,595,5,978824268
4,1,938,4,978301752


In [12]:
train_df.shape

(750121, 4)

In [13]:
test_df.shape

(250088, 4)

### Data Structure Creation
- To prepare the training/test data, we need to create training/test sets in array format with each row representing a user and each cell in the row representing the rating for each movie. This is the expected input for RBM.

In [14]:
num_users = int(max(max(train[:, 0]), max(test[:, 0])))
num_movies = int(max(max(train[:, 1]), max(test[:, 1])))
print("Number of User are:", num_users)
print("Number of Movies are:", num_movies)

Number of User are: 6040
Number of Movies are: 3952


#### We create a function for data conversion which returns a list of lists. Each child list represents one user’s ratings for all movies. If the user did not rate a movie, initialize the rating with 0

In [15]:
def convert(data):
    new_data = []
    for id_users in range(1, num_users + 1):
        ##id of movies that is rated by current users
        id_movies = data[:,1][data[:,0] == id_users]
        
        ##rate of movies that is given by current user
        id_ratings = data[:,2][data[:,0] == id_users]
        
        #inialize ratings for all movies
        #set 0 for movies that are not rated by current users
        ratings = np.zeros(num_movies)
        #movie id starts from 1, 1st movie will be 1st element in rating with index as 0
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [16]:
training_set = convert(train)
test_set = convert(test)

In [17]:
pd.DataFrame(training_set)

,0,1,2,3,4,5,6,7,8,9,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pd.DataFrame(test_set)

,0,1,2,3,4,5,6,7,8,9,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert the list of list type into Tensor 
- Because we will use Pytorch to build the RBM.

In [19]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)
print(training_set)
print(test_set)

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [20]:
print(training_set.shape)
print(test_set.shape)

torch.Size([6040, 3952])
torch.Size([6040, 3952])


### Binary data conversion
- Our task is to predict if users like the movies as 1 and not like as 0.
- RBM will take the movie ratings by a user and try to predict movies that were not rated by the user. - Since the to-be-predicted ratings are computed from the original input, we must keep the input ratings and predicted ratings in a consistent manner.
- Specifically, rating set as 0 previously is reset to be -1, movies given 1 or 2 are set as 0 (not like), and movies rating over 3 set as 1 (like).

In [21]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [22]:
print(training_set)

tensor([[ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.]])


In [23]:
print(test_set)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


###  RBM Architecture

In [40]:
#Model Building
#Inside the __init__ function, we will initialize all parameters that need to be optimized
class RBM():
    def __init__(self, nv, nh):      #nh-number of hidden nodes, nv-number of visible noded
        self.W = torch.randn(nh, nv) #weights for the visible nodes and hidden nodes
        self.a = torch.randn(1, nh)  #a-bias for the probability of hidden nodes given visible node
        self.b = torch.randn(1, nv)  # b-bias for the probability of visible nodes given hidden nodes
        
#Hidden node sampling function
#This function is about sampling hidden nodes given the probabilities of visible nodes.
    def sample_h(self, x): #x-value of visible nodes
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v =torch.sigmoid(activation) #p_h_given_v-the probability of h given v is the sigmoid activation of v
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
#Visible node sampling function
#This function is about sampling Visible nodes given the probabilities of hidden nodes.
    def sample_v(self, y): #y-value of hidden node
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h =torch.sigmoid(activation) #p_v_given_h-is the probabilities of each visible node equal to 1 (being activated).
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
#Contrastive divergence function
#This function update the weights, the bias for visible nodes, and for hidden nodes using the algorithm outlined in this paper
#v0-input vector containing the ratings of all movies by a user
#vk-visible nodes obtained after k samplings from visible nodes to hidden nodes
#ph0-vector of probabilities of hidden node equal to one at the first iteration given v0
#phk-probabilities of hidden nodes given visible nodes vk at the kth iteration.
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
        
#Object creation
nv = len(training_set[0]) #number of movies
nh = 100 ##number of hidden nodes or num of features
batch_size = 100
rbm = RBM(nv, nh)

### Train the RBM model

In [41]:
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    ##loss function
    train_loss = 0
    #normalize the loss, define a counter
    s = 0.
    #implement a batch learning, 
    for id_user in range(0, num_users - batch_size, batch_size):
        #input batch values
        vk = training_set[id_user: id_user+batch_size]
        #target used for loss mesarue: rating 
        v0 = training_set[id_user: id_user+batch_size]
        ##initilize probablity
        #pho: given real rating at begining, probablity of hidden nodes
        ph0, _ = rbm.sample_h(v0)
        #k step of constrative divergence
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            #training on rating that do exist, rating as -1
            vk[v0<0] = v0[v0<0]
        phk, _ = rbm.sample_h(vk)
        #update weights and bias
        rbm.train(v0, vk, ph0, phk)
        #update train loss
        train_loss += torch.mean(torch.abs(v0[v0>0]-vk[v0>0]))
        s += 1
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.1554)
epoch: 2 loss: tensor(0.1370)
epoch: 3 loss: tensor(0.1362)
epoch: 4 loss: tensor(0.1365)
epoch: 5 loss: tensor(0.1365)
epoch: 6 loss: tensor(0.1362)
epoch: 7 loss: tensor(0.1358)
epoch: 8 loss: tensor(0.1361)
epoch: 9 loss: tensor(0.1365)
epoch: 10 loss: tensor(0.1356)


### Test the RBM model

In [47]:
test_loss = 0
s = 0.
for id_user in range(num_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2073)


### Recommendation of movie for giver users

In [48]:
def movie_recos(id_user):
    input = training_set[id_user:id_user+1]
    target = test_set[id_user:id_user+1]
    if len(target[target>=0]) > 0:
        _,h = rbm.sample_h(input)
        _,output = rbm.sample_v(h)
        movie_ratings = pd.DataFrame(output.cpu().detach().numpy()).T
        movie_ratings.columns = ['Pred_Ratings']
        target_ratings = pd.DataFrame(target.cpu().detach().numpy()).T
        target_ratings.columns = ['Actual_Ratings']
        Final_Movie_recos = pd.concat([movie_ratings, target_ratings], axis=1)
        Final_Movie_recos['movie_id'] = Final_Movie_recos.index
        Final_Movie_recos = Final_Movie_recos[Final_Movie_recos.Actual_Ratings==-1]
        Final_Movie_recos = Final_Movie_recos[Final_Movie_recos.Pred_Ratings==1]
        return Final_Movie_recos

### Recommend Moview for user_id 100

In [54]:
Final_Movie_recos = movie_recos(100)
Final_Movie_recos.head()

,Pred_Ratings,Actual_Ratings,movie_id
0,1.0,-1.0,0
1,1.0,-1.0,1
2,1.0,-1.0,2
3,1.0,-1.0,3
4,1.0,-1.0,4


In [55]:
pd.merge(Final_Movie_recos, movies, on='movie_id', how='inner')

,Pred_Ratings,Actual_Ratings,movie_id,movie_name,genre
0,1.0,-1.0,1,Toy Story (1995),Animation|Children's|Comedy
1,1.0,-1.0,2,Jumanji (1995),Adventure|Children's|Fantasy
2,1.0,-1.0,3,Grumpier Old Men (1995),Comedy|Romance
3,1.0,-1.0,4,Waiting to Exhale (1995),Comedy|Drama
4,1.0,-1.0,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...
2624,1.0,-1.0,3947,Get Carter (1971),Thriller
2625,1.0,-1.0,3948,Meet the Parents (2000),Comedy
2626,1.0,-1.0,3949,Requiem for a Dream (2000),Drama
2627,1.0,-1.0,3950,Tigerland (2000),Drama


#### Above are the movie's which user_id 100 may like.